In this first notebook, we will go from the installation of the [PyNN](http://neuralensemble.org/docs/PyNN/index.html) neural desriptor to the display of the simulation of Izhikevich neurons.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
fig_width = 15
phi = (np.sqrt(5)+1)/2

# A selection of Izhikevich neurons

using nengo: https://www.nengo.ai/


In [ ]:
import nengo
from nengo.utils.ensemble import tuning_curves
from nengo.utils.matplotlib import rasterplot

with nengo.Network(seed=0) as model:
    u = nengo.Node(lambda t: np.sin(2 * np.pi * t))
    ens = nengo.Ensemble(10, dimensions=1, neuron_type=nengo.Izhikevich())
    nengo.Connection(u, ens)

with model:
    out_p = nengo.Probe(ens, synapse=0.03)
    spikes_p = nengo.Probe(ens.neurons)
    voltage_p = nengo.Probe(ens.neurons, 'voltage')
    recovery_p = nengo.Probe(ens.neurons, 'recovery')

with nengo.Simulator(model) as sim:
    sim.run(1.0)

t = sim.trange()
plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(t, sim.data[out_p])
plt.ylabel("Decoded output")
ax = plt.subplot(2, 1, 2)
rasterplot(t, sim.data[spikes_p], ax=ax)
plt.ylabel("Neuron");

In [ ]:
def izh_plot(sim):
    t = sim.trange()
    plt.figure(figsize=(12, 10))
    plt.subplot(4, 1, 1)
    plt.plot(t, sim.data[out_p])
    plt.ylabel("Decoded output")
    plt.xlim(right=t[-1])
    ax = plt.subplot(4, 1, 2)
    rasterplot(t, sim.data[spikes_p], ax=ax)
    plt.ylabel("Neuron")
    plt.xlim(right=t[-1])
    plt.subplot(4, 1, 3)
    plt.plot(t, sim.data[voltage_p])
    plt.ylabel("Voltage")
    plt.xlim(right=t[-1])
    plt.subplot(4, 1, 4)
    plt.plot(t, sim.data[recovery_p])
    plt.ylabel("Recovery")
    plt.xlim(right=t[-1])


u.output = 0.2
with nengo.Simulator(model) as sim:
    sim.run(0.1)
izh_plot(sim)

# A selection of Izhikevich neurons

using brian: http://briansimulator.org/


Configure the simulator:

In [ ]:
from brian2 import *


In [ ]:
a = 0.02/ms; b = 0.2/ms
eqs = '''dv/dt = (0.04/ms/mV)*v**2+(5/ms)*v+140*mV/ms-w + I : volt
         dw/dt = a*(b*v-w) : volt/second
         I : volt/second'''
P = NeuronGroup(400, eqs, threshold='v>-50*mV', reset='v=-60*mV')
P.v = -20*mV
Pe = P[:320]
Pi = P[320:]
Ce = Synapses(Pe, P, on_pre='I+=200*mV/second')
Ce.connect(p=0.02)
Ci = Synapses(Pi, P, on_pre='I-=9*mV/second')
Ci.connect(p=0.02)
M = SpikeMonitor(P)
run(1*second)
plot(M.t/ms, M.i, '.')

# using pyNN

adapted from http://neuralensemble.org/docs/PyNN/examples/Izhikevich.html

In [ ]:
import pyNN.brian2 as sim
sim.setup(timestep=0.01, min_delay=0.1)
neuron_type = sim.Izhikevich()
cells = sim.Population(1, neuron_type)

dc_source = sim.DCSource(amplitude=0.5, start=25, stop=50)
ac_source = sim.ACSource(start=75, stop=125, amplitude=0.5, offset=0.25, frequency=100.0, phase=0.0)
noisy_source = sim.NoisyCurrentSource(mean=0.5, stdev=0.05, start=150, stop=175, dt=1.0)
step_source = sim.StepCurrentSource(times=[200, 225, 250], amplitudes=[0.4, 0.6, 0.2])

cells[0].inject(dc_source)
cells[0].inject(ac_source)
cells[0].inject(noisy_source)
cells[0].inject(step_source)

dc_source.record()
ac_source.record()
noisy_source.record()
step_source.record()

cells.record(['v'])

sim.run(275.0)

In [ ]:
if '--plot-figure' in sys.argv:
    import matplotlib.pyplot as plt
    plt.ion()

    plt.figure(figsize=(8,8))

    plt.subplot(2,1,1)
    vm = cells.get_data().segments[0].filter(name="v")[0]
    plt.plot(vm.times, vm, 'r')
    plt.xlabel("time (ms)")
    plt.ylabel("Vm (mV)")

    plt.subplot(2,1,2)
    i_t_dc, i_amp_dc = dc_source._get_data()
    i_t_ac, i_amp_ac = ac_source._get_data()
    i_t_noisy, i_amp_noisy = noisy_source._get_data()
    i_t_step, i_amp_step = step_source._get_data()
    plt.plot(i_t_dc, i_amp_dc, 'k')
    plt.plot(i_t_ac, i_amp_ac, 'k')
    plt.plot(i_t_noisy, i_amp_noisy, 'k')
    plt.plot(i_t_step, i_amp_step, 'k')
    plt.xlabel("time (ms)")
    plt.ylabel("current (nA)")

    plt.suptitle(simulator_name, fontsize=15)
    plt.show(block=True)  # SA: changed
end()

In [ ]:
import pyNN.brian2 as sim
#import pyNN.spinnaker as sim
sim.setup(timestep=0.01, min_delay=1.0)

In [ ]:
# === Build and instrument the network =======================================
spike_source = sim.Population(1, sim.SpikeSourceArray(spike_times=np.arange(10.0, 51, 1)))

neurons = sim.Population(3, sim.Izhikevich(a=0.02, b=0.2, c=-65, d=6, i_offset=[0.014, 0.0, 0.0]))

connection = sim.Projection(spike_source, neurons[1:2], sim.OneToOneConnector(),
                            sim.StaticSynapse(weight=3.0, delay=1.0),
                            receptor_type='excitatory'),

# perturb
electrode = sim.DCSource(start=2.0, stop=92.0, amplitude=0.014)
electrode.inject_into(neurons[2:3])

# record
neurons.record(['v'])  # , 'u'])
neurons.initialize(v=-70.0, u=-14.0)

In [ ]:
# === Run the simulation =====================================================

sim.run(100.0)

In [ ]:
# === plot a figure =============================

from pyNN.utility.plotting import Figure, Panel
data = neurons.get_data().segments[0]
v = data.filter(name="v")[0]
#u = data.filter(name="u")[0]
fig = Figure(
    Panel(v, ylabel="Membrane potential (mV)", xticks=True,
          xlabel="Time (ms)", yticks=True),
    #Panel(u, ylabel="u variable (units?)"),    
)
# === Clean up and quit ========================================================

sim.end()